In [1]:
#|default_exp requisicoes

In [2]:
#|export
import re
import os
from typing import Union, Iterable
from time import time
import urllib3


import requests
from fastcore.basics import store_attr, listify
from fastcore.foundation import L
from fastcore.parallel import parallel
from fastcore.xtras import partialler
from validate_docbr import CPF, CNPJ
from dotenv import load_dotenv, find_dotenv

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
load_dotenv(find_dotenv())


d:\Applications\Scoop\apps\miniconda3\current\envs\receitaws\lib\site-packages\fastprogress\fastprogress.py:107: UserWarning: Couldn't import ipywidgets properly, progress bar will use console behavior
  warn("Couldn't import ipywidgets properly, progress bar will use console behavior")


True

## Constantes

In [3]:
#|export
BASEURL = "https://apinet{}/informacaoeconhecimento/entidade/receitaws/rest/1.0.0/"
TIPOS = {"cpf": "obterPessoaFisica", "cnpj": "obterPessoaJuridica"}
CPF_SIMPLES = (
    "cpf",
    "nome",
    "nomeMae",
    "dataNascimento",
    "estrangeiro",
    "tituloEleitor",
    "dataAtualizacao",
    "dataRegistroAnatel",
    "resultado",
    "unidadeAdministrativaCodigo",
    "anoObito",
    "erro",
)
SEXO = {1: "Masculino", 2: "Feminino", 9: "Não encontrado"}
TAMANHO = {11: "cpf", 14: "cnpj"}
TOKEN_URL_DS = "https://apinetds/token"
TOKEN_URL_HM = "https://apinethm/token"
TOKEN_URL_PD = "https://api.anatel.gov.br/token"
PAYLOAD = {"grant_type": "client_credentials"}
CURL = "curl -k -X POST '{}' -d 'grant_type=client_credentials' -H 'Authorization: Basic {}'"
REQUEST = (
    "curl -k -X 'GET' '{}' -H 'accept: application/json' -H 'Authorization: Bearer {}'"
)
AMBIENTE = {
    "ds": ("ds", TOKEN_URL_DS, "KEY_DS"),
    "hm": ("hm", TOKEN_URL_HM, "KEY_HM"),
    "pd": ("", TOKEN_URL_PD, "KEY_PD"),
}


## Classe Principal para Requisição

In [4]:
#|export
class Requisicao:
    """Classe para fazer requisições ao Web Service Receita - WS da Anatel, este encapsula o Infoconv da Receita Federal"""

    def __init__(
        self,
        cpf_usuario: str | int,  # CPF do usuário requisitante
        tipo: str,  # Tipo de Requisição CPF | CNPJ
        origem: str,  # Texto com identificação da requisição: e.g. "Teste"
        ambiente: str = "hm",  # Ambiente onde realizar a requisição: ds: desenvolvimento, hm: homologação, pd: produção
        cache: int = 36,  # Tempo de expiração do cache em meses (Produção)
    ):
        store_attr()
        self._validar_parametros()
        self.origem = self.origem[:30]
        self.cpf_usuario = "".join(re.findall("\d", str(self.cpf_usuario)))
        self.credentials = {}

    def _validar_parametros(self) -> None:
        """Método interno de verificação e validação de parâmetros"""
        if (
            not isinstance(self.tipo, str)
            or (tipo := self.tipo.lower()) not in TIPOS.keys()
        ):
            raise ValueError(
                "Forneça uma string válida para o tipo de requisição: {CPF | CNPJ} (Case insensitive)"
            )
        self.tipo = tipo
        self.classe = CPF() if self.tipo == "cpf" else CNPJ()
        assert isinstance(
            self.origem, str
        ), "origem não pode ficar vazio e deve ser uma string de até 30 caracteres"
        assert (
            ambiente := self.ambiente.lower()
        ) in AMBIENTE.keys(), (
            f"Ambiente inválido, escolha uma das opções {AMBIENTE.keys()}"
        )
        self._definir_ambiente(*AMBIENTE[ambiente])

    def _definir_ambiente(
        self,
        ambiente: str,  # Ambiente onde realizar a requisição: hm: homologação, pd: produção
        token: str,  # Url para requisitar o token
        key: str,  # Identificador do tipo de chave de acordo com o ambiente
    ) -> None:
        """Define as variáveis token e chave de acordo com o ambiente"""
        self.ambiente = ambiente
        self.token_url = token
        self.key = key

    def _pad_zeros(
        self,
        identificador: int
        | str,  # Identificador da requisição: CPF ou CNPJ de acordo com o tipo
    ) -> str:
        """Preenche o identificador com zeros à esquerda até o comprimento total: {CPF:11, CNPJ:14}"""
        identificador = "".join(re.findall("\d", str(identificador)))
        if self.tipo == "cpf":
            return identificador.zfill(11)
        elif self.tipo == "cnpj":
            return identificador.zfill(14)
        return ""

    def gerar_token(self) -> None:
        """Gera o token de requisição com validade de 1h à partir da chave de acesso do usuário
        As chaves de acesso são acessadas como variáveis de ambiente: KEY_HM | KEY_PD
        """
        if start := self.credentials.get("time"):
            if time() - start < 3600 and "token" in self.credentials:
                return
        headers = {"Authorization": f"Basic {os.getenv(self.key)}"}
        r = requests.post(self.token_url, json=PAYLOAD, verify=False, headers=headers)
        if (status := r.status_code) != 200:
            raise ValueError(f"A requisição de token não foi bem-sucedida: {status}")
        self.credentials["token"] = r.json().get("access_token")
        self.credentials["time"] = time()

    def _formatar_url(
        self,
        identificador: int
        | str,  # Identificador da requisição: CPF ou CNPJ de acordo com o tipo
    ) -> str:  # Url formatada para a requisição GET REST
        """Recebe o identificador CPF|CNPJ e formata retorna a url formatada da requisição"""
        req_TIPO = TIPOS[self.tipo]
        suffix = ""
        if self.cache is not None:
            try:
                cache = int(self.cache)
                assert (
                    cache >= 0
                ), "Tempo de expiração do cache inválido, escolha um número inteiro maior que zero"
            except ValueError as e:
                raise ValueError(
                    "Valor inválido de expira_cache, escolha um número inteiro maior que zero"
                ) from e
            req_TIPO += "IgnoraCacheAntigo"
            suffix = f"&mesesExpiraCache={cache}"

        return f"{BASEURL.format(self.ambiente)}{req_TIPO}?{self.tipo}={identificador}&cpfUsuario={self.cpf_usuario}&origem={self.origem}{suffix}"

    @staticmethod
    def nivelar_json(
        json_dict: dict,  # Dicionário no formato json retornado pela requisição
    ) -> dict:  # Dicionário nivelado para formato tabular
        """Recebe um dicionário json com diferentes níveis e retorna os registros em um único nível"""
        d = {}
        for k, v in json_dict.items():
            if isinstance(v, list):
                v = "|".join(v)
            if isinstance(v, dict):
                d |= {f"{k}.{sk}": sv for sk, sv in v.items()}
                continue
            d[k] = v
        return {
            k: v.lstrip().rstrip() if isinstance(v, str) else v for k, v in d.items()
        }

    @staticmethod
    def _get_request(
        url: str,  # Url formatada da requisição
        token: str,  # Token de autorização
    ) -> dict:
        """Efetua a requisição na url and retorna a resposta json"""
        headers = {"accept": "application/json", "Authorization": f"Bearer {token}"}
        r = requests.get(url, headers=headers, verify=False)
        if r.status_code == 200 and r.headers["content-type"] == "application/json":
            return Requisicao.nivelar_json(r.json())
        return {}

    def consultar(
        self,
        identificador: Union[
            str, int
        ],  # Identificador da requisição: CPF ou CNPJ de acordo com o tipo
    ) -> dict:  # Dicionário com o resultado da requisição
        """Efetua a requisição do identificador e retorna um dicionário"""
        identificador = self._pad_zeros(identificador)
        if not self.classe.validate(identificador):
            raise ValueError(
                f"{self.tipo.upper()} Inválido! Verifique o identificador digitado: {identificador}"
            )
        self.gerar_token()
        return Requisicao._get_request(
            self._formatar_url(identificador), self.credentials["token"]
        )

    def validar_identificadores(
        self,
        identificadores: Iterable[
            Union[str, int]
        ],  # Lista com os identificadores: CPF ou CNPJ
    ) -> L:
        """Completa com zeros os identificadores e os filtra retornando somente os válidos"""
        return (
            L(listify(identificadores))
            .map(self._pad_zeros)
            .filter(self.classe.validate)
        )

    def consultar_em_lote(
        self,
        identificadores: Iterable[
            Union[str, int]
        ],  # Lista com os identificadores: CPF ou CNPJ
        n_workers: int = 8,  # Número de requisições a serem efetuadas em paralelo
    ) -> Iterable[dict]:  # Lista com os dicionários resultantes das requisições
        """Efetua a requisição da lista de ids identificadores de forma paralelizada por threads
        Filtra os identificadores válidos primeiramente antes de fazer a requisição.
        Os identificadores não retornados foram considerados inválidos
        """
        identificadores = self.validar_identificadores(identificadores)
        if not identificadores:
            return L()
        self.gerar_token()
        urls = identificadores.map(self._formatar_url)
        func = partialler(Requisicao._get_request, token=self.credentials["token"])
        return parallel(func, urls, n_workers=n_workers, threadpool=True, progress=True)